In [26]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import os

def procesar_datos_y_guardar_txt(archivo_entrada, archivo_salida):
    # Leer los datos del archivo y omitir las primeras 7 líneas
    datos = np.loadtxt(archivo_entrada, skiprows=7)

    # Extraer la columna de interés (columna 6, índice 5 en Python)
    senal = datos[:, 5]
    
    for dato in senal:
        dato=float(dato)
    
    # Crear un vector de tiempo (asumiendo que los datos son muestreados uniformemente)
    num_muestra = np.arange(len(senal))

    # Calcular el vector de tiempo en segundos
    fs=1000
    tiempo = (num_muestra / fs)*1000 #ms
    
    # Calcular la señal en milivoltios
    senalV = (((senal / 1024) - 0.5) * 3.3)/1009
    senalmV = senalV*1000
    senaluV = senalV*1000000
    senal_para_enviar = senalmV

    # Guardar tiempo y senal en un archivo TXT con encabezados y valores separados por espacios
    
    with open(archivo_salida, 'w') as txtfile:
        txtfile.write("timestamp,accY\n")  # Escribir los encabezados
        for t, s in zip(tiempo, senal_para_enviar):
            txtfile.write(f"{t},{s}\n")
    
def enviar_a_edgeimpulse(archivo_txt, api_key, etiqueta):

    # Add the files you want to upload to Edge Impulse
    files = [archivo_txt]

    # # Replace the label with your own.
    label =  etiqueta

    # Create a list of tuples for the 'files' parameter
    file_tuples = [('data', (os.path.basename(file), open(file, 'rb'), 'application/octet-stream')) for file in files]

    # Upload the file to Edge Impulse using the API, and print the response.
    res = requests.post(url='https://ingestion.edgeimpulse.com/api/training/files',
                        headers={
                            'x-label': label,
                            'x-api-key': api_key,
                        },
                        # Creating the data payload for the request.
                        files=file_tuples
                        )

    if (res.status_code == 200):
        print('Uploaded file(s) to Edge Impulse\n', res.status_code, res.content)
    else:
        print('Failed to upload file(s) to Edge Impulse\n',
            res.status_code, res.content)

# Ejemplo de uso
archivo_entrada = r"C:\ISB\data\cesar_pesta.txt"
archivo_salida = r"C:\ISB\eeg-pestaneo.txt"  # Nombre del archivo de salida en formato TXT
api_key = 'ei_b53b7673e25c9423e64efc72588ebe09dba64bfe6c5f73a6'
etiqueta = 'EEG pestaneando'

# Procesar datos y guardar en TXT
procesar_datos_y_guardar_txt(archivo_entrada, archivo_salida)

# Enviar archivo TXT a Edge Impulse
enviar_a_edgeimpulse(archivo_salida, api_key, etiqueta)


Uploaded file(s) to Edge Impulse
 200 b'{"success":true,"files":[{"success":true,"projectId":431563,"sampleId":1059432266,"fileName":"eeg-pestaneo.txt.50rkiq71.json"}]}'
